In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_json('dataset.json')
df

,1179055004553900032_twitter,1179063826874032128_twitter,1178793830532956161_twitter,1179088797964763136_twitter,1179085312976445440_twitter,13851720_gab,24198545_gab,14567516_gab,19679648_gab,15858421_gab,...,9981407_gab,9981575_gab,9982674_gab,9984358_gab,9988840_gab,9989999_gab,9990225_gab,9991681_gab,9992513_gab,9998729_gab
post_id,1179055004553900032_twitter,1179063826874032128_twitter,1178793830532956161_twitter,1179088797964763136_twitter,1179085312976445440_twitter,13851720_gab,24198545_gab,14567516_gab,19679648_gab,15858421_gab,...,9981407_gab,9981575_gab,9982674_gab,9984358_gab,9988840_gab,9989999_gab,9990225_gab,9991681_gab,9992513_gab,9998729_gab
annotators,"[{'label': 'normal', 'annotator_id': 1, 'targe...","[{'label': 'normal', 'annotator_id': 1, 'targe...","[{'label': 'normal', 'annotator_id': 4, 'targe...","[{'label': 'hatespeech', 'annotator_id': 1, 't...","[{'label': 'hatespeech', 'annotator_id': 4, 't...","[{'label': 'hatespeech', 'annotator_id': 1, 't...","[{'label': 'hatespeech', 'annotator_id': 4, 't...","[{'label': 'offensive', 'annotator_id': 4, 'ta...","[{'label': 'hatespeech', 'annotator_id': 4, 't...","[{'label': 'hatespeech', 'annotator_id': 4, 't...",...,"[{'label': 'hatespeech', 'annotator_id': 208, ...","[{'label': 'offensive', 'annotator_id': 206, '...","[{'label': 'hatespeech', 'annotator_id': 200, ...","[{'label': 'offensive', 'annotator_id': 239, '...","[{'label': 'offensive', 'annotator_id': 199, '...","[{'label': 'offensive', 'annotator_id': 217, '...","[{'label': 'offensive', 'annotator_id': 220, '...","[{'label': 'offensive', 'annotator_id': 206, '...","[{'label': 'hatespeech', 'annotator_id': 209, ...","[{'label': 'hatespeech', 'annotator_id': 200, ..."
rationales,[],[],[],"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,...","[[1, 1, 1, 1, 0, 0], [1, 1, 0, 1, 0, 0], [1, 1...","[[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,"[[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 1, 1, 1], [0, 1, 1, 1]]","[[0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...",[],"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,..."
post_tokens,"[i, dont, think, im, getting, my, baby, them, ...","[we, cannot, continue, calling, ourselves, fem...","[nawt, yall, niggers, ignoring, me]","[<user>, i, am, bit, confused, coz, chinese, p...","[this, bitch, in, whataburger, eating, a, burg...","[laura, loomer, raped, me, while, screaming, a...","[and, this, is, why, i, end, up, with, nigger,...","[nogs, jews, and, dykes, how, enriching]","[guilty, until, proven, innocent, unless, you,...","[tired, i, can, not, support, abortion, from, ...",...,"[<user>, congrats, on, making, the, kike, hate...","[wow, she, comes, right, out, and, says, the, ...","[more, and, more, leftists, are, moving, close...","[a, negro, honey, trap]","[this, vile, racist, ghetto, ass, dumber, than...","[if, ur, still, on, twitter, tell, carlton, i,...","[when, i, first, got, on, here, and, said, i, ...","[was, macht, der, moslem, wenn, der, zion, geg...","[it, is, awful, look, at, world, demographics,...","[the, jewish, globalist, elite, have, only, im..."


In [3]:
temp = df.values

In [4]:
idxs = np.array([i for i, x in enumerate(temp[0]) if 'gab' in x])
len(idxs)

11093

In [5]:
ids = np.array([int(x[:-4]) for x in temp[0, idxs]])
ids

array([13851720, 24198545, 14567516, ...,  9991681,  9992513,  9998729])

In [6]:
targets = []
for rows in temp[1,idxs]:
    labels = [rows[0]['label'], rows[0]['label'], rows[0]['label']]
    counts = np.array([0] * 3)
    for label in labels:
        if label == 'normal':
            counts[0] += 1
        elif label == 'offensive':
            counts[1] += 1
        else:
            counts[2] += 1
    if counts.max() == 1:
        targets.append(1)
    else:
        targets.append(counts.argmax())
targets = np.array(targets)
len(targets), sum(targets==0), sum(targets==1), sum(targets==2)
# targets[targets>1]=1
# len(targets), sum(targets==0), sum(targets==1)

(11093, 2456, 3498, 5139)

In [7]:
data = np.array([' '.join(x) for x in temp[3,idxs]])
data

array(['laura loomer raped me while screaming at me in her disgusting kike language and said we must exterminate the goyim laura loomer loomergate',
       'and this is why i end up with nigger trainee doctors who can not speak properly lack basic knowledge of biology it truly scary if the public only knew',
       'nogs jews and dykes how enriching', ...,
       'was macht der moslem wenn der zion gegen seinen propheten hetzt was machst du wenn die roten ratten gegen deinen toten opa hetzen',
       'it is awful look at world demographics asians are fucking everywhere another betrayal white genocide is real',
       'the jewish globalist elite have only imported a few million muslims to multiculturalize weaken america right now most all violence against lgbt is by black street criminals who are sometimes lgbt themselves and random as muslim grow i think we will start seeing organized attacks'],
      dtype='<U526')

In [8]:
df = pd.DataFrame()
df['id'] = ids
df['text'] = data
df['target'] = targets
df

,id,text,target
0,13851720,laura loomer raped me while screaming at me in...,2
1,24198545,and this is why i end up with nigger trainee d...,2
2,14567516,nogs jews and dykes how enriching,1
3,19679648,guilty until proven innocent unless you are a ...,2
4,15858421,tired i can not support abortion from a moral ...,2
...,...,...,...
11088,9989999,if ur still on twitter tell carlton i said his...,1
11089,9990225,when i first got on here and said i hate trump...,1
11090,9991681,was macht der moslem wenn der zion gegen seine...,1
11091,9992513,it is awful look at world demographics asians ...,2


In [14]:
df.to_csv("hatexplain.csv", index=None)

In [9]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=2021)

In [10]:
train_data = train_df.values[:,1:]
val_data = val_df.values[:,1:]
vectorizer = TfidfVectorizer()
vectorizer.fit(data)
x_train = vectorizer.transform(train_data[:,0])
y_train = np.asarray(train_data[:,1], dtype=int)
x_val = vectorizer.transform(val_data[:,0])
y_val = np.asarray(val_data[:,1], dtype=int)

In [11]:
clf = LogisticRegression(max_iter=10000)
clf.fit(x_train, y_train)

LogisticRegression(max_iter=10000)

In [12]:
print(classification_report(clf.predict(x_val), y_val))

              precision    recall  f1-score   support

           0       0.29      0.50      0.37       295
           1       0.44      0.47      0.45       659
           2       0.78      0.63      0.69      1265

    accuracy                           0.56      2219
   macro avg       0.50      0.53      0.51      2219
weighted avg       0.61      0.56      0.58      2219

